
## Welcome to the IDWR Storage Credit Use Tracker

This Notebook is used to __estimate__ the amount of storage credits consumed per month by individual items or all items owned by a user. This calculator takes a snapshot of an item or a user's items at runtime and estimates the cost in credits to store each item per month. It cannot account for items that change size. 

This calculator assumes that hosted feature layer __views__ are assigned a storage cost of 0 credits. This is because technically, Esri does not charge for the hosting of feature layer views, since the underlying data actually belongs to the hosted feature layer from which the view was created. The views have a size of 0 Bytes, but the ArcGIS API for Python functions read the hosted feature layer's attachments as if they belong to the view (which they don't). This was leading to wildly inaccurate and even "negative" credit consumption for individual items and users. 


### Run the following code block to login to ArcGIS Online and prepare this notebook's functions 


In [ ]:
# Libraries
from arcgis.gis import GIS
import pandas as pd
import math
from getpass import getpass
import tempfile


# TODO: Make this something the user can change
output_path = tempfile.gettempdir() #r"C:\Temp"

# Get user credentials
# TODO: This could just use the user's current active portal
username = input("Please enter your ArcGIS Online username: ")
password = getpass("Please enter your password: ")
portal = 'https://idwr.maps.arcgis.com'
#currentDate = datetime.now()


# Define functions
def estimateItemCreditCost(idForItem):
    ''' This function calculates the monthly storage credits spent on an item.
    
      It initially attempted to calculate the lifetime cost of an item.
      An assumption that an item's size remained constant from creation until the instant the function is called had to be made, but this was unreliable and unfair. 
      An hourly rate can still be determined from the monthly rate.'''
    # Get item info from AGOL 
    item = gis.content.get(idForItem)
    
    if item.type == "Feature Service":
        try:
            totalSize = item.size
             # Get total size of just feature attachments and calculate credit rate, per size stored
            fa = item.layers[0]
            totalAttachmentsSize = 0.0

            offsetVals = [0, 1000, 2000, 3000, 4000, 5000]

            for val in range(len(offsetVals)):
                attachmentList = fa.attachments.search(as_df=False, offset=offsetVals[val])
                faSize = sum(each['SIZE'] for each in attachmentList)
                totalAttachmentsSize += faSize
        
            faSizeGB = round(totalAttachmentsSize/(1<<30), 3)
  
            # Get feature attachment size, credit rate, per size stored to calculate cost
            faCreditRate = 1.2
            faPerStored = 1
            faCost = faCreditRate * (faSizeGB/faPerStored)

            # The difference is the size of the feature layer itself, without attachments
            flSize = totalSize - totalAttachmentsSize
            # Get feature layer size, credit rate, per size stored to calculate cost
            flSizeMB = round(flSize/(1<<20), 3)
            flCreditRate = 2.4
            flPerStored = 10
            flCost = flCreditRate * (flSizeMB/flPerStored)

            itemTotalCost = faCost + flCost
            # if itemTotalCost is negative, it is just set to zero. 
            # This means it's a view, which does not consume any credits.
            if itemTotalCost < 0:
                itemTotalCost = 0
                return itemTotalCost
            else:
                return itemTotalCost
        except:
            # Feature layer storage cost equation
            size = item.size/(1<<20)
            creditRate = 2.4
            perStored = 10
            itemTotalCost = creditRate * (size/perStored)
            return itemTotalCost
    else:
        # Get item size in GB, credit rate, per GB stored
        size = round(item.size/(1<<30), 3)
        creditRate = 1.2
        perStored = 1
        itemTotalCost = creditRate * (size/perStored)
        return itemTotalCost


# Log into portal
gis = GIS(portal, username, password)
me = gis.users.me
print("Logged in to ArcGIS Online as: ")
me




### Individual Item

#### Run the following block if you would like to calculate the monthly storage cost of an individual item

This will prompt you for an item's 32-digit hash ID and then print out credits consumed per month based on the size of the item and the size of the item's attachments (if applicable). The hash ID is used instead of searching by name as it is the best unique identifier for specifying exactly which item you'd like to calculate storage credit costs.


In [ ]:
# Identify item whose storage cost you would like to tally

cst = estimateItemCreditCost(input("Enter the item's 32-digit hash ID: "))
print("This item uses " + str(round(cst, 3)) + " credits per month.")


### User Items

#### Run the following block if you would like to estimate the monthly storage cost of a user's items

This will prompt you for an IDWR employee's ArcGIS Online username and print out the number of items they own as well as the credits they consume per month. 

This will also provide a more in-depth breakdown of a user's monthly credit consumption per item in a CSV. You can take the 32-digit hash ID from the second column to find and view the item in ArcGIS Online.

In [ ]:
# Identify a user whose storage credit usage you would like to tally
userOfInterest = gis.users.get(input("Enter the username of a member of your ArcGIS Online Organization: "))
name = userOfInterest.username

# Get a list of their stuff
itemsList = gis.content.search(query=f'owner:{name}', max_items=10000) # Need to set max pretty high, default too low
print("Estimating credits used...")

itemIdList = []
itemSizeList = []
indivItemCost = []
totalCost = 0.0

for each_item in itemsList:
    item_size = each_item.size
    item_id = each_item.id
    itemIdList.append(item_id)
    itemSizeList.append(item_size)
    ucst = estimateItemCreditCost(item_id)
    indivItemCost.append(ucst)
    totalCost = totalCost + ucst

creditById = pd.DataFrame({
    'id':itemIdList,
    'creditCost':indivItemCost,
    'size':itemSizeList
})
out_file_path = f"{output_path}\\{name}.csv"
creditById.to_csv(path_or_buf=out_file_path)
    
print(f"{name} uses {round(totalCost, 3)} credits per month over {len(itemsList)} items.")
print(f"Detailed results in '{out_file_path}'")

### Group Items and Storage Cost CSV

#### Run to get amount of credits consumed by a group per month, as well as output a CSV

This block operates nearly identically to the above User Items and Item and Storage Cost CSV blocks. 


In [ ]:
# This block added at Melissa Madden's request, 03 March 2023.

# Identify a group whose storage credit usage you would like to tally
groupOfInterest = gis.groups.get(input("Enter the 32-digit hash ID of a group in your ArcGIS Online Organization: "))

# Get a list of their stuff
gItemsList = groupOfInterest.content()
print(f"Group contains {len(gItemsList)} items.")
print("Estimating credits used...")

gItemIdList = []
gItemSizeList = []
gItemCost = []  

for i in range(len(gItemsList)):
    id = gItemsList[i].id
    gItemIdList.append(id)
    groupItemSize = gItemsList[i].size
    gItemSizeList.append(groupItemSize)
    gcst = estimateItemCreditCost(gItemIdList[i])
    gItemCost.append(gcst)


gTotalCost = sum(gItemCost)
    
print(f"The '{groupOfInterest.title}' group uses {round(gTotalCost, 3)} credits per month.")


# Spit out CSV for an individual user to look at wonkyness
creditById = pd.DataFrame({
    'id':gItemIdList,
    'size':gItemSizeList,
    'creditCost':gItemCost
})
out_file_path = f"{output_path}\\{groupOfInterest.title}.csv"
creditById.to_csv(path_or_buf=out_file_path)
print(f"Results in '{out_file_path}'")